# Data Expo 2009 - Airline on-time performance
## by (mahmoud lotfi)

## Investigation Overview

> Describe the overall goals of your presentation here.

## Dataset Overview

> Provide a brief overview of the dataset to be presented here.

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

In [ ]:
# reset seaborn settings
sns.reset_orig()
# set plotting color
base_color = sns.color_palette()[0]

In [ ]:
# load in the dataset into a pandas dataframe


> Note that the above cells have been set as "Skip"-type slides. That means
that when the notebook is rendered as http slides, those cells won't show up.

## 01. Which Airlines have the longest delays

> WN was the worste airline for departures delay, while AS,NW was the worst for arrivals.

In [ ]:
longest = flights[flights['ArrDelay']>0].groupby('UniqueCarrier')\
                .agg(DepDelayMean=('DepDelay', 'mean'), ArrDelayMean=('ArrDelay', 'mean'))\
                .sort_values('ArrDelayMean', ascending=False)

In [ ]:
plt.figure(figsize=(10,7))
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)
longest.plot.bar(ax=ax);
plt.ylabel('Mintes')
plt.grid()

## 02.What is the worst time of day to travel?
> the worst dealy around 06 AM

In [ ]:
flights['hours'] = pd.to_datetime(flights['CRSDepTime'], format='%I:%M %p').dt.hour
flights['minutes'] = pd.to_datetime(flights['CRSDepTime'], format='%I:%M %p').dt.minute
flights = flights.sort_values(['hours', 'minutes'])
flights = flights.drop(columns=['minutes'])
df = flights[(flights['ArrDelay'] > 15)].groupby('hours')\
                .agg(ArrDelayCount=('ArrDelay', 'count'), ArrDelaysum=('ArrDelay', sum)) 
df = df.reset_index()

In [ ]:
df['ArrDelayAvg'] = (df['ArrDelaysum']/df['ArrDelayCount']).round(2)

In [ ]:
sns.lineplot(data = df, x='hours', y='ArrDelayCount');
plt.grid()

## 03. Flight delay reasons and how much they contribute to the number of delayed flights

>We also see that the weather is not the main reason for delays. Weather only contributes to 2% of the delays.


In [ ]:
df_row = pd.merge(flights.groupby(['UniqueCarrier']).agg(UniqueCarrierCount=('UniqueCarrier', 'count')), 
                  flights[(flights['ArrDelay'] > 15)].groupby(['UniqueCarrier']).agg(ArrDelaycount=('ArrDelay', 'count')),
                  on='UniqueCarrier', how='inner')

In [ ]:
df_delay = flights[(flights['ArrDelay'] > 15)]

In [ ]:
df_delay = df_delay[(flights['LateAircraftDelay'] != 0) | (flights['SecurityDelay'] != 0) |
        (flights['NASDelay'] != 0) | (flights['WeatherDelay'] != 0) | 
        (flights['CarrierDelay'] != 0) | (flights['WeatherDelay'] != 0)]

In [ ]:
df_delay['LateAircraftDelay'] = df_delay['LateAircraftDelay'].apply(lambda x: 1 if x else 0)
df_delay['SecurityDelay'] = df_delay['SecurityDelay'].apply(lambda x: 1 if x else 0)
df_delay['NASDelay'] = df_delay['NASDelay'].apply(lambda x: 1 if x else 0)
df_delay['WeatherDelay'] = df_delay['WeatherDelay'].apply(lambda x: 1 if x else 0)
df_delay['CarrierDelay'] = df_delay['CarrierDelay'].apply(lambda x: 1 if x else 0)

In [ ]:
df_delay = df_delay.groupby(['UniqueCarrier']).agg(
    # Get count of the column for each group
    LateAircraftDelayCount=('LateAircraftDelay', sum),
    # Get count of the column for each group
    SecurityDelayCount=('SecurityDelay', sum),
    # Get count of the column for each group
    NASDelayCount=('NASDelay', sum),
    # Get count of the column for each group
    WeatherDelayCount=('WeatherDelay', sum),
    # Get count of the column for each group
    CarrierDelayCount=('CarrierDelay', sum),   
)

In [ ]:
df_delay = pd.merge(df_delay, df_row,  on='UniqueCarrier', how='inner')

In [ ]:
df_delay['pct'] = (df_delay['ArrDelaycount'].astype(float)/df_delay['UniqueCarrierCount'].astype(float)).round(2)

In [ ]:
wethear_sum   = df_delay['WeatherDelayCount'].sum()
nas_sum       = df_delay['NASDelayCount'].sum()
carrier_sum   = df_delay['CarrierDelayCount'].sum()
security_sum  = df_delay['SecurityDelayCount'].sum()
aircraft_sum  = df_delay['LateAircraftDelayCount'].sum()
#ArrDelaycount_sum = df_delay['ArrDelaycount'].sum()

num_of_overall = wethear_sum + nas_sum + carrier_sum + security_sum + aircraft_sum

wethear_sum  = (wethear_sum/num_of_overall).round(2)*100
nas_sum      = (nas_sum/num_of_overall).round(2)*100
carrier_sum  = (carrier_sum/num_of_overall).round(2)*100
security_sum = (security_sum/num_of_overall).round(2)*100
aircraft_sum = (aircraft_sum/num_of_overall).round(2)*100

plt.figure(figsize=(10,7))
X = ['weather delays' , 'nas delays','carrier delays','late air craft','secuirity delays']
Y = [wethear_sum, nas_sum, carrier_sum, aircraft_sum, security_sum]
plt.pie(Y, labels=X , explode=[0.09,0,0,0,0.3],  autopct='%1.2f%%');
plt.title('Contribution of each Delay reason to overall delays');

> Once you're ready to finish your presentation, check your output by using
nbconvert to export the notebook and set up a server for the slides. From the
terminal or command line, use the following expression:
> > `jupyter nbconvert <file_name>.ipynb --to slides --post serve --template output_toggle`

> This should open a tab in your web browser where you can scroll through your
presentation. Sub-slides can be accessed by pressing 'down' when viewing its parent
slide. Make sure you remove all of the quote-formatted guide notes like this one
before you finish your presentation!